# 3D Particle Structure Generator Exploration
This notebook demonstrates generating particles from the VAE model connected to the virtual environment.

In [ ]:
import torch
import matplotlib.pyplot as plt
from models.vae import VAE
from data.generator import generate_particles
from utils.metrics import compute_pairwise_distances

In [ ]:
# Setup config mapped to model defaults
n_particles = 50
input_dim = n_particles * 3
hidden_dim = 128
latent_dim = 16

model = VAE(input_dim, hidden_dim, latent_dim)
model.load_state_dict(torch.load('model.pth', weights_only=True))
model.eval()
print('Model loaded!')

In [ ]:
# Generate references
orig_pts = generate_particles(n_particles, min_distance=0.08)

with torch.no_grad():
    z = torch.randn(1, latent_dim)
    gen_pts = model.decode(z).view(n_particles, 3)

In [ ]:
fig = plt.figure(figsize=(10, 5))

ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(orig_pts[:, 0], orig_pts[:, 1], orig_pts[:, 2], alpha=0.6, c='b')
ax1.set_title('Original Data')
ax1.set_xlim(0,1); ax1.set_ylim(0,1); ax1.set_zlim(0,1)

ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(gen_pts[:, 0], gen_pts[:, 1], gen_pts[:, 2], alpha=0.6, c='r')
ax2.set_title('VAE Generated')
ax2.set_xlim(0,1); ax2.set_ylim(0,1); ax2.set_zlim(0,1)

plt.show()

In [ ]:
orig_dists = compute_pairwise_distances(orig_pts)
gen_dists = compute_pairwise_distances(gen_pts)

plt.hist(orig_dists.numpy(), bins=30, alpha=0.5, label='Original')
plt.hist(gen_dists.numpy(), bins=30, alpha=0.5, label='Generated')
plt.title('Pairwise Distance Histogram')
plt.xlabel('Distance')
plt.legend()
plt.show()